In [1]:
import json
import re

In [2]:
with open("./rosmsgs.json", "r") as jsonFile:
    messages_data = json.load(jsonFile)
messages = sorted(messages_data.keys())

In [3]:
from msginfo import *

In [4]:
def collect_modules(messages):
    modules = {}
    for msg in messages:
        mi = MsgInfo(msg, messages_data[msg])
        if mi.module not in modules:
            modules[mi.module] = {}
        msgdata = { 
            'info'    : mi, 
            'depends' : mi.get_local_dependencies()
        }    
        modules[mi.module][mi.name] = msgdata  
    return modules

modules = collect_modules(messages)

In [5]:
# Deduplicating identifiers  
module = modules['Geometry_msgs'].copy()

for mod_name in modules:
    module = modules[mod_name]
    # Gather duplicate field names
    fields = {}
    for msg in module:
        for f in module[msg]['info'].get_field_names():
            if f not in fields:
                fields[f] = []
            fields[f].append(msg)

    duplicates = {}
    for field, msgs in fields.iteritems():
        if len(msgs) < 2: 
            continue
        for msg in msgs:
            if msg not in duplicates:
                duplicates[msg] = []
            duplicates[msg].append(field)

    # Apply prefix for duplicates
    for msg, fields in duplicates.iteritems():
        module[msg]['info'].prefix_fields(fields)


In [6]:
folder = "../imandra_model/src-messages"
for mod_name in modules:
    module = modules[mod_name].copy()
    code, generated = [], set()
    while module:
        for m in module:
            deps = module[m]['depends']
            deps = set(deps) - generated 
            if not deps:
                code.append(module[m]['info'].gen_typedecl())
                generated.add(m)
        for m in generated:
            if m in module:
                del module[m]
    
    filename = folder + "/" + mod_name.lower() + ".ml"
    with open(filename, "w") as mlfile:
        mlfile.write("open Basic_types;;\n\n")
        for c in code:
            mlfile.write(c)
            mlfile.write("\n\n")

In [14]:
def get_type_parser(tinfo, current_module=None):
    tdecl = "json_to_" + kw(tinfo['name']) + "_opt"
    if 'module' in tinfo and tinfo['module'] != current_module:
        tdecl = "{0}_of_json.{1}".format( tinfo['module'] , tdecl )
    if tinfo.get('list', False):
        tdecl = "(jlist {})".format(tdecl)
    return tdecl

def gen_json_serializer(self):
        if self.entries == []:
            return "let json_to_{0}_opt json : {0} = Some ()".format(self.name)
        code = ["let json_to_{}_opt json = ".format(self.name)]
        for entry in self.entries:
            etype = get_type_parser(entry['type'], self.module)
            code.append("    jreq json \"{0}\" {1} @@ fun {0} ->".format(entry['name'], etype))
        code += ["    Some {"]
        for entry in self.entries:
            code.append("    {0};".format(entry['name']))
        code += ["    }"]
        
        return "\n".join(code)

In [15]:
folder = "../imandra_model/src-messages-pp"
for mod_name in modules:
    module = modules[mod_name].copy()
    code, generated = [], set()
    while module:
        for m in module:
            deps = module[m]['depends']
            deps = set(deps) - generated 
            if not deps:
                code.append(gen_json_serializer(module[m]['info']))
                generated.add(m)
        for m in generated:
            if m in module:
                del module[m]
    
    filename = folder + "/" + mod_name.lower() + "_of_json.ml"
    with open(filename, "w") as mlfile:
        mlfile.write("open Json_utils;;\n")
        mlfile.write("open Basic_types_of_json;;\n")        
        mlfile.write("open Ros_messages.{};;\n\n".format(mod_name))        
        for c in code:
            mlfile.write(c)
            mlfile.write("\n\n")